# Configurations

## Configure Stanza

In pgBackRest, a **stanza** is a configuration unit that defines a specific PostgreSQL cluster. 

It encapsulates all the settings and parameters related to that cluster, such as its database paths, backup locations, and other related configurations.

A stanza allows pgBackRest to manage multiple PostgreSQL clusters on the same or different servers, each with its own distinct configuration.

```BASH
# Create a configuration file (INI format) with configurations for a single db cluster called "stanza" in pgBackRest jargon
BACK_REST_CONFIG_FILE=/etc/pgbackrest/pgbackrest.conf
mkdir -p /etc/pgbackrest/
chown -R postgres:postgres /etc/pgbackrest/
printf '[demo]
pg1-path=/var/lib/postgresql/16/demo\n' > $BACK_REST_CONFIG_FILE
```

## Configure Repository

Most probably the repository in production will be on an another server to avoid crashing together with the host.

A single machine setup could be possible in production only if machine is being backed up with a different backup tool on a file system level.

### Create Repository

```BASH
# Add repo path
mkdir -p /var/lib/pgbackrest
chmod 750 /var/lib/pgbackrest
chown postgres:postgres /var/lib/pgbackrest

# Add repo path to demo stanza
printf '\n[global]
repo1-path=/var/lib/pgbackrest\n' >> $BACK_REST_CONFIG_FILE
```

### Configure WAL Archiving On Server

```BASH
# Configure (details in WAL archiving lesson)
sed -i "s/#*archive_command.*/archive_command = 'pgbackrest --stanza=demo archive-push %p'/" $DB_CONF_FILE
sed -i "s/#*archive_mode.*/archive_mode = on/" $DB_CONF_FILE
sed -i "s/#*max_wal_senders.*/max_wal_senders = 3/" $DB_CONF_FILE
sed -i "s/#*wal_level.*/wal_level = replica/" $DB_CONF_FILE
# Check
grep "^archive_command\|^archive_mode\|^max_wal_senders\|^wal_level" $DB_CONF_FILE
```

### Configure Archive Push Command

Configurations for the pgbackrest `archive-push` used to push WAL files on archiving trigger (WAL file is switched)

```BASH
# Add compression to archived WAL files
printf '\n[global:archive-push]
compress-level=3\n' >> $BACK_REST_CONFIG_FILE
```

### Configure Backup Retention

To empty disk space we are implementing a retention policy that deletes unneeded backups

It's always a good idea to keep as mush backups as possible to enhance our point in time recovery window

There are 2 types of retention policies:
1. Time based - backups older than x time are deleted if there is at least one backup that is new enough
1. Count based - whenever the backups number exceed the count the oldest are deleted

Retention Policy is enforced by the `expire` command which is called automatically every time a backup is made successfully, but it can also be ran by the user

```BASH
# Add count based retention
sed -i "s/^\[global\].*/[global]\nrepo1-retention-full=2/" $BACK_REST_CONFIG_FILE
```

### Configure Repo Encryption

Since the repository basically contains all over production data and even more (historic data as well) it better be secured with encryption!

It's better to encrypt on client side (pgbackrest side) even when the server supports built in encryption (all of the cloud storages are) to make sure that nothing happens in the network between them

```BASH
# Add count based retention
ENCRYPTION_KEY=$(openssl rand -base64 48)
sed -i "s/^\[global\].*/[global]\nrepo1-cipher-type=aes-256-cbc/" $BACK_REST_CONFIG_FILE
sed -i 's,^\[global\].*,[global]\nrepo1-cipher-pass='"$ENCRYPTION_KEY"',' $BACK_REST_CONFIG_FILE
```

### Don't Wait For Checkpoint

By default pgBackRest is waiting for a normally scheduled `CHECKPOINT` on PG server, for the demo purposes we will make it create a `CHECKPOINT` for it's purposes

Even in production, in most cases, it's a good idea to turn on this option to be sure that the backup is done in time. Only in very busy servers it can be problematic for performance since the backup routine should be scheduled probably to once a day or so 

In [ ]:
sed -i "s/^\[global\].*/[global]\nstart-fast=y/" $BACK_REST_CONFIG_FILE

## Setup Stanza

After configuring both the stanza and the repository it's going to be backed up in, it's time to create the stanza object that is actually used when running a backup.

### Create stanza

In [ ]:
pg_ctlcluster 16 demo start
su postgres -c "pgbackrest --stanza=demo --log-level-console=info stanza-create"

### Check Configurations

This checks that the configurations for the stanza created and the repositories are valid and that WAL archiving is working as expected

It actually really archives a WAL file by forcing the server to switch wal with `pg_switch_wal()`